# TaskBuddy: Intelligent Voice Memo Task Manager 🎤📝✨

Welcome to the **TaskBuddy** project! This notebook serves as the high-level table of contents and an architectural guide for our intelligent voice memo task manager.

TaskBuddy is being built with a strong emphasis on clean architecture, **SOLID principles**, **Test-Driven Development (TDD)**, and robust **Dependency Injection (DI)**. Our goal is to create a task management system that goes beyond conventional tools by leveraging AI to understand and organize tasks based on individual user's language and habits.

Use this notebook as your entry point to understand the project's structure, its core components, and how each layer contributes to the overall vision.

In [4]:
import sys
import os
from loguru import logger # Using Loguru's global logger

# Add the project root to the Python path to allow importing modules
# This assumes the notebook is in 'taskbuddy_project/notebooks/'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Configure Loguru logging for this notebook session (using dev mode for verbose output)
# The import itself often triggers setup_logging, but calling it explicitly ensures settings are applied.
import config.loguru_setup # This import triggers Loguru setup
from config.config import DEBUG_MODE # For displaying debug mode

# Call setup_logging explicitly (though import already triggers it in loguru_setup.py)
# This can be useful if you need to pass specific overrides, but currently redundant.
config.loguru_setup.setup_logging() # Ensure Loguru setup is explicitly applied

logger.info(f"INFO - Notebook environment initialized. Project modules are now importable.")
logger.info(f"INFO - Python interpreter: {sys.executable}")
logger.info(f"INFO - Current working directory: {os.getcwd()}")
logger.info(f"INFO - Project root added to PATH: {project_root}")
logger.info(f"INFO - Running in DEBUG_MODE: {DEBUG_MODE}")

# A simple check to ensure a core module can be imported
try:
    from task import Task
    logger.success(f"SUCCESS - Successfully imported 'task.Task'. Core modules are accessible.")
except ImportError as e:
    logger.error(f"ERROR - Failed to import core module: {e}. Check PYTHONPATH or environment activation.")


INFO     | config.loguru_setup | Logging to file: c:\Users\jarde\Projects\aura\aura-data\logs\taskbuddy.log
INFO     | config.loguru_setup | Loguru setup complete. Running in dev mode.
INFO     | __main__ | INFO - Notebook environment initialized. Project modules are now importable.
INFO     | __main__ | INFO - Python interpreter: c:\Users\jarde\Anaconda3\envs\aura-data\python.exe


INFO     | __main__ | INFO - Current working directory: c:\Users\jarde\Projects\aura\aura-data\notebooks
INFO     | __main__ | INFO - Project root added to PATH: c:\Users\jarde\Projects\aura\aura-data
INFO     | __main__ | INFO - Running in DEBUG_MODE: dev
SUCCESS  | __main__ | SUCCESS - Successfully imported 'task.Task'. Core modules are accessible.


---
## Project Structure and Layers

This project is organized into distinct architectural layers, each with its own responsibilities. Click on the links below to navigate to the detailed documentation notebooks for each section.

* [**1. Data Layer Overview**](01_Data_Layer_Overview.ipynb)
    * _Focus:_ How task data is stored, retrieved, and managed (e.g., CSV, database interactions).
    * _Key Components:_ `ICrudRepository`, `BaseCsvRepository`, `CsvTaskRepository`, `sample_data.csv`.

* [**2. Business Layer Overview**](02_Business_Layer_Overview.ipynb)
    * _Focus:_ The core logic of the TaskBuddy application, independent of data storage or UI.
    * _Key Components:_ `TaskManagerService`, `Task` model.

* [**3. Core Infrastructure**](03_Core_Infrastructure.ipynb)
    * _Focus:_ Foundational services and architectural patterns like Dependency Injection and Centralized Logging.
    * _Key Components:_ `DependencyContainer`, `Loguru`.

* [**4. User Interface (UI) Layer**](04_UI_Layer_Overview.ipynb)
    * _Focus:_ How users interact with TaskBuddy (e.g., CLI, Web App, Mobile App).
    * _Key Components:_ (To be developed)

* [**5. AI & Voice Memo Features**](05_AI_Voice_Memo_Features.ipynb)
    * _Focus:_ Integration of voice memo recording, AI parsing for categorization, and intelligent output.
    * _Key Components:_ (To be developed)

* [**6. Testing Strategy**](06_Testing_Strategy.ipynb)
    * _Focus:_ In-depth look at our Test-Driven Development (TDD) approach and testing utilities.
    * _Key Components:_ `pytest`, test fixtures.

* [**7. Extending TaskBuddy: Adding New Objects & Data Sources**](07_Extending_TaskBuddy.ipynb)
    * _Focus:_ The codified methodology for introducing new domain objects and integrating them with diverse data sources, ensuring architectural consistency.
    * _Key Components:_ `ICrudRepository`, `BaseCsvRepository`, new domain objects and services.

---

# Extending TaskBuddy: Adding New Objects & Data Sources

This notebook outlines the disciplined methodology for introducing new domain objects and their corresponding data sources into the **TaskBuddy** project. Adhering to these steps ensures that our architecture remains **consistent, extensible, and maintainable**, upholding our commitment to SOLID principles.

This serves as a living blueprint for scaling TaskBuddy to manage any type of entity (e.g., `Weather`, `Reminder`, `Traffic`) from diverse data sources.

---

## Methodology for Introducing a New Object and Data Source

When a new domain object (e.g., `Weather`, `Reminder`) is introduced into the TaskBuddy project, along with its persistence, the following steps must be strictly followed:


### Phase 1: Defining the Core Object (Domain Model Layer)

1.  **Define the New Object Class:**
    * **Location:** `business/` (e.g., `business/weather.py`)
    * **Purpose:** This class (`Weather`, `Reminder`, etc.) will hold the data structure and any intrinsic behaviors (like `mark_complete` for `Task`) for the new entity.
    * **Key Attributes:** Must include a `uuid.UUID` `id` for unique identification, and any other relevant properties.
    * **Example (for `Weather`):**
        ```python
        # business/weather.py
        import uuid
        from datetime import datetime # Example import
        # ... other imports as needed

        class Weather:
            def __init__(self, city: str, temperature: float, timestamp: datetime, weather_id: uuid.UUID = None):
                self.id = weather_id if weather_id else uuid.uuid4()
                self.city = city
                self.temperature = temperature
                self.timestamp = timestamp
                
            def __repr__(self):
                return f"Weather(id='{self.id}', city='{self.city}', temp={self.temperature}°F)"

            def __eq__(self, other):
                if not isinstance(other, Weather):
                    return NotImplemented
                return self.id == other.id

            def __hash__(self):
                return hash(self.id)
        ```


### Phase 2: Data Access Layer Implementation

2.  **No Changes to `ICrudRepository.py`:**
    * **Location:** `business/interfaces/ICrudRepository.py`
    * **Purpose:** This interface remains generic and **closed for modification**. It defines the fundamental `add`, `get_by_id`, `get_all`, `update`, `delete` methods that apply to *any* entity.

3.  **Create a Concrete Repository for the New Object:**
    * **Location:** `data/` (e.g., `data/csv_weather_repository.py`)
    * **Purpose:** This concrete class will handle the specific CSV (or other data source) interactions for the *new object type*.
    * **Inheritance:** It **must inherit from `BaseCsvRepository[NewObject]`** (e.g., `BaseCsvRepository[Weather]`). This ensures it gets all the generic CSV handling logic for free.
    * **Implementation:** It **must implement the abstract methods from `BaseCsvRepository`**:
        * `_to_dict(self, entity: NewObject) -> Dict[str, Any]`: Converts an instance of the new object into a dictionary suitable for a CSV row.
        * `_from_dict(self, row: Dict[str, str]) -> NewObject`: Converts a dictionary (CSV row) back into an instance of the new object.
    * **Logging:** It will directly `from loguru import logger` for all its logging, ensuring consistency with our global Loguru setup.
    * **Example (for `CsvWeatherRepository`):**
        ```python
        # data/csv_weather_repository.py
        import uuid
        from typing import Dict, Any
        from datetime import datetime # Example import
        from data.base_csv_repository import BaseCsvRepository
        from business.weather import Weather # New Weather object
        from loguru import logger # Consistent global logger

        class CsvWeatherRepository(BaseCsvRepository[Weather]):
            def __init__(self, file_path: str = None):
                # Resolve path as done in CsvTaskRepository
                import os
                current_dir = os.path.dirname(os.path.abspath(__file__))
                project_root = os.path.dirname(os.path.dirname(current_dir))
                resolved_file_path = file_path if file_path else os.path.join(project_root, 'data', 'csv', 'weather_data.csv')
                
                super().__init__(file_path=resolved_file_path, entity_name="weather")
                self._expected_headers = ['id', 'city', 'temperature', 'timestamp'] # Specific to Weather
                logger.debug("DEBUG - CsvWeatherRepository initialized.")

            def _to_dict(self, weather: Weather) -> Dict[str, Any]:
                return {
                    'id': str(weather.id),
                    'city': weather.city,
                    'temperature': weather.temperature,
                    'timestamp': weather.timestamp.isoformat() # Convert datetime to string
                }

            def _from_dict(self, row: Dict[str, str]) -> Weather:
                try:
                    weather_id = uuid.UUID(row['id'])
                    city = row['city']
                    temperature = float(row['temperature'])
                    timestamp = datetime.fromisoformat(row['timestamp']) # Convert string to datetime
                    return Weather(city=city, temperature=temperature, timestamp=timestamp, weather_id=weather_id)
                except Exception as e:
                    logger.error(f"ERROR - Failed to convert row to Weather: {e}. Row: {row}", exc_info=False)
                    raise ValueError(f"Invalid weather data: {row}. Error: {e}")
        ```


### Phase 3: Business Service Layer Implementation

4.  **Create a New Business Service for the Object:**
    * **Location:** `business/` (e.g., `business/weather_service.py`)
    * **Purpose:** This service encapsulates the business logic related to the new object type.
    * **Dependency:** Its constructor **must accept an `ICrudRepository[NewObject]`** (e.g., `weather_repository: ICrudRepository[Weather]`). This demonstrates **Dependency Inversion Principle (DIP)**.
    * **Methodology:** Its methods will call the generic CRUD operations (e.g., `weather_repository.add(new_weather_obj)`) on its injected repository.
    * **Logging:** It will directly `from loguru import logger` for all its logging.
    * **Example (for `WeatherService`):**
        ```python
        # business/weather_service.py
        from typing import List
        from business.interfaces.ICrudRepository import ICrudRepository
        from business.weather import Weather # New Weather object
        from loguru import logger # Consistent global logger
        from datetime import datetime # Example import

        class WeatherService:
            def __init__(self, weather_repository: ICrudRepository[Weather]):
                self._weather_repository = weather_repository
                logger.debug("DEBUG - WeatherService initialized.")

            def get_current_weather_for_city(self, city_name: str) -> List[Weather]:
                logger.info(f"INFO - Getting weather for {city_name}.")
                all_weather = self._weather_repository.get_all()
                return [w for w in all_weather if w.city.lower() == city_name.lower()]

            def record_new_weather_reading(self, city: str, temperature: float) -> Weather:
                new_reading = Weather(city=city, temperature=temperature, timestamp=datetime.now())
                self._weather_repository.add(new_reading)
                logger.success(f"SUCCESS - Recorded new weather: {city}, {temperature}°F")
                return new_reading
        ```


### Phase 4: Dependency Injection Configuration

5.  **Register the New Repository with the `DependencyContainer`:**
    * **Location:** `main.py` (within the `configure_dependencies` function)
    * **Purpose:** This wires up the new interface-to-implementation mapping.
    * **Action:** Add a `container.register()` call.
    * **Example (in `main.py`'s `configure_dependencies`):**
        ```python
        # main.py (within configure_dependencies)
        from business.interfaces.ICrudRepository import ICrudRepository
        from task import Task
        from business.weather import Weather # New Weather object import
        # ...
        container.register(ICrudRepository[Task], "data.csv_task_repository.CsvTaskRepository")
        container.register(ICrudRepository[Weather], "data.csv_weather_repository.CsvWeatherRepository") # New line for Weather
        ```


### Phase 5: Testing the New Object (Test Layer)

6.  **Create a Dedicated Test File for the New Repository:**
    * **Location:** `tests/` (e.g., `tests/test_csv_weather_repository.py`)
    * **Purpose:** Unit test the new concrete repository implementation.
    * **Methodology:** Follow TDD: write failing tests for `add`, `get_by_id`, `get_all`, `update`, `delete` (for the `Weather` object), then implement them. Use a temporary CSV file fixture for isolation, just like `test_csv_task_repository.py`. Since `BaseCsvRepository` implements the generic CRUD logic, the tests for the new repository will look very similar to `test_csv_task_repository.py`, but asserting `Weather` object properties.

7.  **Create a Dedicated Test File for the New Business Service (Optional, but Recommended):**
    * **Location:** `tests/` (e.g., `tests/test_weather_service.py`)
    * **Purpose:** Unit test the business logic of the new service.
    * **Methodology:** Use `unittest.mock` (or `pytest-mock`) to mock the `ICrudRepository[Weather]` dependency injected into `WeatherService`. This isolates the service logic from data persistence details.


### Phase 6: Interactive Documentation (Jupyter Lab)

8.  **Create a New Jupyter Notebook:**
    * **Location:** `notebooks/` (e.g., `07_Extending_TaskBuddy.ipynb`)
    * **Purpose:** Document this entire methodology for adding new objects and data sources. Provide executable examples (though the examples above might require setting up dummy files or mocking for pure notebook execution).
    * **Action:** Add a link to this new notebook in `project.ipynb`'s Table of Contents.
        * Update `project.ipynb`'s Table of Contents in the "Project Structure and Layers" section with:
            ```markdown
            * [**7. Extending TaskBuddy: Adding New Objects & Data Sources**](07_Extending_TaskBuddy.ipynb)
                * _Focus:_ The codified methodology for introducing new domain objects and integrating them with diverse data sources.
                * _Key Components:_ `ICrudRepository`, `BaseCsvRepository`, new domain objects and services.
            ```
